In [1]:
!pip install flask diffusers transformers torch accelerate
!pip install pyngrok
!pip install fastapi uvicorn pyngrok
!pip install python-multipart
!pip install nest_asyncio



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok

# Add your ngrok authentication token
!ngrok authtoken 2pUNMdn5heZDqr50ndLJ9hFA348_7BNec91SNvazozmDtzkf5


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


This Code is For

This Code is For desired Prompt

This is Fixated only For Dresses as Prompt

In [3]:
import nest_asyncio
nest_asyncio.apply()


This Is Integrating with Fast API

In [ ]:
from fastapi import FastAPI, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.middleware.cors import CORSMiddleware
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch
import os
import uuid
from pyngrok import ngrok

# Initialize FastAPI app
app = FastAPI()

# Add CORS middleware to allow external access (optional)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Directory to save output images
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load Stable Diffusion model
model_path = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
pipe.enable_attention_slicing()
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Expose static files (to serve generated images)
app.mount("/static", StaticFiles(directory=OUTPUT_DIR), name="static")

@app.get("/", response_class=HTMLResponse)
async def home():
    return """
    <html>
        <head><title>Stable Diffusion</title></head>
        <body>
            <h1>Generate Stylish Dress Images</h1>
            <form action="/generate-image" method="post">
                <label for="prompt">Explain Your Mood or Scenario for the event you want Dresses For:</label><br>
                <input type="text" id="prompt" name="prompt" style="width: 300px;" placeholder="Enter your mood or event"><br><br>

                <label for="gender">Select Gender:</label><br>
                <input type="radio" id="male" name="gender" value="Male" checked>
                <label for="male">Male</label><br>
                <input type="radio" id="female" name="gender" value="Female">
                <label for="female">Female</label><br><br>

                <button type="submit">Generate Stylish Dresses</button>
            </form>
        </body>
    </html>
    """

@app.post("/generate-image", response_class=HTMLResponse)
async def generate_image(prompt: str = Form(...), gender: str = Form(...)):
    try:
        # Append gender-specific text to the prompt
        if gender == "Male":
            prompt = f"Only Dresses and costumes For Men for the Event {prompt}."
        else:
            prompt = f"Only Dresses and costumes For Women for the Event {prompt}."

        num_variants = 5  # Generate 5 images
        images = pipe([prompt] * num_variants, num_inference_steps=25, height=384, width=384).images

        # Save all images and collect their paths
        image_paths = []
        for img in images:
            filename = f"{uuid.uuid4()}.png"
            image_path = os.path.join(OUTPUT_DIR, filename)
            img.save(image_path)
            image_paths.append(image_path)

        # Display all images on the result page
        image_tags = ''.join([f'<img src="/static/{os.path.basename(path)}" alt="Image" style="max-width: 300px; margin: 10px;">' for path in image_paths])
        return f"""
        <html>
            <head><title>Generated Images</title></head>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    background-color: #f4f4f4;
                    margin: 0;
                    padding: 0;
                    display: flex;
                    flex-direction: column;
                    align-items: center;
                }}
                h1 {{
                    color: #333;
                    margin-top: 20px;
                }}
                .container {{
                    display: flex;
                    flex-wrap: wrap;
                    justify-content: center;
                    gap: 20px;
                    padding: 20px;
                }}
                .image-card {{
                    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                    border-radius: 8px;
                    overflow: hidden;
                    background-color: #fff;
                    width: 300px;
                    text-align: center;
                }}
                .image-card img {{
                    width: 100%;
                    border-bottom: 1px solid #ddd;
                }}
                .buttons {{
                    margin: 20px 0;
                }}
                .buttons a {{
                    text-decoration: none;
                    padding: 10px 20px;
                    background-color: #007bff;
                    color: #fff;
                    border-radius: 5px;
                    margin: 0 5px;
                    transition: background-color 0.3s;
                }}
                .buttons a:hover {{
                    background-color: #0056b3;
                }}
            </style>
            <body>
                <h1>Here Are Your Generated Designs!</h1>
                <div class="container">
                    {image_tags}
                </div>
                <div class="buttons">
                    <a href="/">Generate More Images</a>
                </div>
            </body>
        </html>
        """
    except Exception as e:
        return f"<h1>Error: {str(e)}</h1>"

# Run FastAPI with ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

...

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:     Started server process [1746]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://f35c-104-196-20-186.ngrok-free.app" -> "http://localhost:8000"
INFO:     2601:c0:8100:8a30:4d67:7dae:71e3:d92a:0 - "GET / HTTP/1.1" 200 OK
INFO:     2601:c0:8100:8a30:4d67:7dae:71e3:d92a:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
